<a href="https://colab.research.google.com/github/ggnanaraj/Story/blob/main/Stock0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install nselib

In [3]:
import nselib
data = nselib.trading_holiday_calendar()
#print(data)

In [5]:
from nselib import capital_market

In [45]:
data=capital_market.bhav_copy_with_delivery("01-03-2024")
print(data.columns)

Index(['SYMBOL', 'SERIES', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE',
       'CLOSE_PRICE', 'PREV_CLOSE', 'TTL_TRD_QNTY', 'TURNOVER_LACS',
       'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER', 'DATE1'],
      dtype='object')


In [ ]:
#data1=data('SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER')
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)

##data1.sort_values(by=['GAP_UP'],ascending=False)
#data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
#data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
#data_VOL=data_5PC_GAP_UP[data_5PC_GAP_UP['TTL_TRD_QNTY'] > 50000] #Quantity greater than 50000
#data_VOL1=data_VOL[data_VOL['DELIV_PER'] != "-"]
##data_VOL1[data_VOL1['DELIV_PER'] > 5]
#data_VOL1.DELIV_PER=data_VOL1.DELIV_PER.astype(float)
#data_VOL1=data_VOL1[data_VOL1['DELIV_PER'] > 20]  #
#data_VOL1=data_VOL1[data_VOL1['DELIV_PER'] < 90]
#data_VOL1['GAP_down']=(((data_VOL1['HIGH_PRICE']-data_VOL1['LOW_PRICE'])/data_VOL1['HIGH_PRICE'])*100).round(2)
#data_VOL1





In [44]:
from nselib import capital_market
data=capital_market.bhav_copy_with_delivery("01-03-2024")
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)
data1['TTL_TRD_QNTY']=0
data1['DELIV_PER']=0

#data1=data1[data1['DELIV_PER'] != "-"]
data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
data_PC=data_5PC_GAP_UP[data_5PC_GAP_UP['PREV_CLOSE'] > 10]
#---------------------------------
dataO=capital_market.bhav_copy_with_delivery("29-02-2024")
data1O=dataO.loc[1:,['SYMBOL','TTL_TRD_QNTY','DELIV_PER']]
#from nselib import capital_market
#data2 = capital_market.price_volume_and_deliverable_position_data(symbol='SBIN', from_date='29-02-2024', to_date='01-03-2024')
#data3=capital_market.equity_list( )
#data1O.set_index('SYMBOL',inplace=True)
#data1O


In [50]:
list_PC=data_PC['SYMBOL'].to_list()
list_PC.sort()
for i in list_PC:
  print(i)
#data_PC.value_counts()
#data_PC.describe()
data2O=data1O[data1O['SYMBOL'].isin(list_PC)]
data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True)

AARTISURF
AMBICAAGAR
CGPOWER
COOLCAPS
DHTL
DOLLEX
HOMESFY
KLL
LANDMARK
LEMERITE
MPSLTD
PREMIERPOL
REMUS
RITEZONE
SADHAV
SAIFL
SASTASUNDR
SGIL
SHAREINDIA
SHYAMTEL
SJLOGISTIC
TOUCHWOOD
WELENT
YCCL
YUDIZ


<ipython-input-50-8d5c1379568d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True)


In [48]:
data_PC.sort_values(by=['SYMBOL'])

,SYMBOL,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRICE,PREV_CLOSE,GAP_UP,TTL_TRD_QNTY,DELIV_PER
63,AARTISURF,348.00,348.00,303.00,325.50,330.00,5.45,0,0
155,AMBICAAGAR,34.50,34.50,30.40,30.80,31.80,8.49,0,0
447,CGPOWER,480.50,494.95,461.00,465.00,443.25,8.40,0,0
496,COOLCAPS,459.00,460.00,445.00,445.00,431.00,6.50,0,0
580,DHTL,223.00,223.00,207.00,209.05,212.35,5.02,0,0
605,DOLLEX,49.00,49.25,45.95,46.40,46.20,6.06,0,0
955,HOMESFY,468.95,468.95,455.00,455.00,437.30,7.24,0,0
1272,KLL,101.55,107.80,97.05,106.40,93.90,8.15,0,0
1330,LANDMARK,777.95,789.95,748.00,758.35,719.65,8.10,0,0
1340,LEMERITE,59.85,59.85,56.15,58.05,56.50,5.93,0,0
